In [18]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from sklearn import svm
from sklearn.svm import SVC
from skrebate import ReliefF
from scipy.stats import zscore
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from itertools import combinations

sns.set_theme(style='white', font_scale=1.8)

plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.autolayout'] = True

# Dataframe collection, parsing and preprocessing
The dataframes (df_rna, df_prottissue, df_protserum, etc.) are created to store the numeric data from each mode, and therefore doesn't include any metadata for each dataset.

The dataframes drop genes/proteins/metabolites that contain one or more NaN.

On the proteomics, since we're interested in the protein-coding genes rather than the proteins themselves, the proteins without a gene name association are dropped in the code below.

In [19]:
# Define the file paths
rna_file_path = '../Data/CCFData/Exercise Proteomics, Metabolomics and RNAseq/3. Exercise Tissue RNAseq/EXERCISE MICE RNA SEQ TISSUE Normalized count.csv'
prottissue_file_path = '../Data/CCFData/Exercise Proteomics, Metabolomics and RNAseq/2. Exercise Tissue Proteomics/Exercise Mice RAW MUSCLE PROTOEMICS .xlsx'
protserum_file_path = '../Data/CCFData/Exercise Proteomics, Metabolomics and RNAseq/1. Exercise Serum Proteomics/Exercise Mice Serum Proteomics RAW DATA.xlsx'

# Read the Excel file and set the first column as the index
df_rna = pd.read_csv(rna_file_path, index_col=0)
df_prottissue = pd.read_excel(prottissue_file_path, engine='openpyxl', index_col=2, skiprows=1)
df_protserum = pd.read_excel(protserum_file_path, sheet_name = 'All data', engine='openpyxl', index_col=2, header=3)

# Replacing gene Ensembl IDs for their names, contained in one of the Differential Expression files
gene_annotation_file_path = "../Data/CCFData/Exercise Proteomics, Metabolomics and RNAseq/3. Exercise Tissue RNAseq/NoExAm_vs_NoExPBS_DEResults_wNormCounts.csv"
gene_annotation_data = pd.read_csv(gene_annotation_file_path)
# Creating and using the Ensembl ID to gene name dictionary
ensembl_to_gene_name_dict = pd.Series(gene_annotation_data.external_gene_name.values, index=gene_annotation_data.Ensembl_ID).to_dict()
df_rna.rename(index=ensembl_to_gene_name_dict, inplace=True)

# Renaming proteomics tissue columns
df_prottissue = df_prottissue.drop(df_prottissue.columns[range(3)], axis=1)
df_prottissue.columns =  (["NE PBS " + str(i) for i in range(1, 9)] + ["NE AmAc " + str(i) for i in range(1, 9)] + ["E PBS " + str(i) for i in range(1, 9)] + ["E AmAc " + str(i) for i in range(1, 9)])

# Dropping NaNs and empty indexed proteins, by removing duplicated indexes (in this case, null strings)
df_protserum = df_protserum.dropna()
df_protserum = df_protserum[~df_protserum.index.duplicated(keep=False)]
df_protserum = df_protserum.drop(df_protserum.columns[range(36, 68)], axis=1)
df_protserum = df_protserum.drop(df_protserum.columns[range(4)], axis=1)

# NE PBS vs. NE AmAc
Here we're setting labels with the following assignment:
- 0 for NE PBS
- 1 for NE AmAc
- 2 for E PBS
- 3 for E AmAc

This notebook performs a multimodal analysis on only the NE PBS and NE AmAc groups, treating it as a binary classification problem, hence the dataframes use the labels and subsequent masks to filter only these two groups.

RNA has its own ordering due to a missing sample (E PBS 6, deemed outlier)

In [20]:
# Since RNA data is missing E PBS 6, it has its own classes
classes_rna = []
for col in df_rna.columns:
    if col.startswith('NE PBS'):
        classes_rna.append(0)
    elif col.startswith('NE AmAc'):
        classes_rna.append(1)
    elif col.startswith('E PBS'):
        classes_rna.append(2)
    elif col.startswith('E AmAc'):
        classes_rna.append(3)

# Create classes arrays for proteomics data (NE PBS, NE Am, E PBS, E AmAc)
classes_prot = []
for col in df_prottissue.columns:
    if col.startswith('NE PBS'):
        classes_prot.append(0)
    elif col.startswith('NE AmAc'):
        classes_prot.append(1)
    elif col.startswith('E PBS'):
        classes_prot.append(2)
    elif col.startswith('E AmAc'):
        classes_prot.append(3)

# Group selection: NE PBS vs NE AmAc

In [21]:
# Each modalitiy has a different ordering, we use different masks for each modality
mask_rna = pd.Series(classes_rna, index=df_rna.columns).isin([0, 1])
mask_prot = pd.Series(classes_prot, index=df_prottissue.columns).isin([0, 1])

In [22]:
# Create the labels for our bivariate classification models dataframe
labels_rna = pd.DataFrame(index=df_rna.columns)
labels_rna['Label'] = df_rna.columns.map(lambda x: 0 if x.startswith('NE PBS') else (1 if x.startswith('NE AmAc') else None))
labels_rna.dropna(inplace=True)
labels_rna = labels_rna.values.ravel()

labels_prot = pd.DataFrame(index=df_prottissue.columns)
labels_prot['Label'] = df_prottissue.columns.map(lambda x: 0 if x.startswith('NE PBS') else (1 if x.startswith('NE AmAc') else None))
labels_prot.dropna(inplace=True)
labels_prot = labels_prot.values.ravel()

In [23]:
# This drops all the other classes
df_rna = df_rna.loc[:,mask_rna.values]
df_prottissue = df_prottissue.loc[:,mask_prot.values]
df_protserum = df_protserum.loc[:,mask_prot.values]

In [24]:
# For transcriptomics, we filter rows where 70% of the values are above 10, transpose, and normalize using z-score
df_rna_normalized = df_rna[df_rna.gt(10).mean(axis=1) > 0.7].T.apply(zscore, axis=0)

# For proteomics and metabolomics, we only transpose, and normalize using z-score
df_prottissue_normalized = df_prottissue.T.apply(zscore, axis=0)
df_protserum_normalized = df_protserum.T.apply(zscore, axis=0)

# Relief parameters

In [25]:
N_reliefF = 100
K = 3

In [26]:
# Select features that separate both classes on transcriptomics
fs = ReliefF(n_neighbors = K, n_features_to_select = N_reliefF, n_jobs = -1, )
fs.fit(df_rna_normalized.values, labels_rna)
df_rna_relief = df_rna_normalized.iloc[:,fs.top_features_[:N_reliefF]]

# Select features that separate both classes on proteomics tissue
fs = ReliefF(n_neighbors = K, n_features_to_select = N_reliefF, n_jobs = -1, )
fs.fit(df_prottissue_normalized.values, labels_prot)
df_prottissue_relief = df_prottissue_normalized.iloc[:,fs.top_features_[:N_reliefF]]

# Select features that separate both classes on proteomics serum
fs = ReliefF(n_neighbors = K, n_features_to_select = N_reliefF, n_jobs = -1, )
fs.fit(df_protserum_normalized.values, labels_prot)
df_protserum_relief = df_protserum_normalized.iloc[:,fs.top_features_[:N_reliefF]]

In [27]:
genes_relief_rna = df_rna_relief.columns
genepairs_relief_rna = pd.DataFrame(combinations(genes_relief_rna, 2), columns=['Gene 1','Gene 2'])

genes_relief_prottissue = df_prottissue_relief.columns
genepairs_relief_prottissue = pd.DataFrame(combinations(genes_relief_prottissue, 2), columns=['Gene 1','Gene 2'])

genes_relief_protserum = df_protserum_relief.columns
genepairs_relief_protserum = pd.DataFrame(combinations(genes_relief_protserum, 2), columns=['Gene 1','Gene 2'])

In [28]:
# MODIFY HERE FOR DESIRED GROUP SELECTION
output_path = '../Output/NEPBSvsNEAmAc/'
if not os.path.exists(output_path):
   os.makedirs(output_path)

In [29]:
# Performance of RNA gene pairs
scores_cv = []
scores_training = []
for pair in tqdm(genepairs_relief_rna.values.tolist()):
    df_to_train = df_rna_relief[pair]

    clf = SVC(kernel='linear', C=100)
    clf.fit(df_to_train.values, labels_rna)

    cv_score = cross_val_score(clf, df_to_train.values, labels_rna, cv=5)

    scores_cv.append(cv_score.mean())
    scores_training.append(clf.score(df_to_train.values, labels_rna))

scores = pd.DataFrame(list(zip(genepairs_relief_rna['Gene 1'].values, genepairs_relief_rna['Gene 2'].values, scores_cv, scores_training)), columns=['Gene 1','Gene 2','cv','train'])
scores.to_csv(output_path + 'bivariate_scores_RNA.csv', index=False)

100%|██████████| 4950/4950 [00:53<00:00, 92.73it/s]


In [30]:
# Performance of proteomics tissue gene pairs
scores_cv = []
scores_training = []
for pair in tqdm(genepairs_relief_prottissue.values.tolist()):
    df_to_train = df_prottissue_relief[pair]

    clf = SVC(kernel='linear', C=100)
    clf.fit(df_to_train.values, labels_prot)

    cv_score = cross_val_score(clf, df_to_train.values, labels_prot, cv=4)

    scores_cv.append(cv_score.mean())
    scores_training.append(clf.score(df_to_train.values, labels_prot))

scores = pd.DataFrame(list(zip(genepairs_relief_prottissue['Gene 1'].values, genepairs_relief_prottissue['Gene 2'].values, scores_cv, scores_training)), columns=['Gene 1','Gene 2','cv','train'])
scores.to_csv(output_path + 'bivariate_scores_PROTTISSUE.csv', index=False)


# Performance of proteomics serum gene pairs
scores_cv = []
scores_training = []
for pair in tqdm(genepairs_relief_protserum.values.tolist()):
    df_to_train = df_protserum_relief[pair]

    clf = SVC(kernel='linear', C=100)
    clf.fit(df_to_train.values, labels_prot)

    cv_score = cross_val_score(clf, df_to_train.values, labels_prot, cv=4)

    scores_cv.append(cv_score.mean())
    scores_training.append(clf.score(df_to_train.values, labels_prot))

scores = pd.DataFrame(list(zip(genepairs_relief_protserum['Gene 1'].values, genepairs_relief_protserum['Gene 2'].values, scores_cv, scores_training)), columns=['Gene 1','Gene 2','cv','train'])
scores.to_csv(output_path + 'bivariate_scores_PROTSERUM.csv', index=False)

100%|██████████| 4950/4950 [00:47<00:00, 104.02it/s]


In [31]:
def plot_pair_SVM(pair, df, labels, path=None, show=False):
    df_to_plot = df[pair]
    # Plot using matplotlib.pyplot
    plt.figure()
    x = df_to_plot[pair[0]].values
    y = df_to_plot[pair[1]].values
    plt.plot(x[labels ==0], y[labels ==0], 'bo', label = 'NE PBS')
    plt.plot(x[labels ==1], y[labels ==1], 'r^', label = 'NE AmAc')
    # Train the classifier
    clf = SVC(kernel='linear', C=100, probability=True)
    clf.fit(df_to_plot.values, labels.ravel())
    # Create a mesh to plot in
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    n_points = 1000
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, n_points),
                            np.linspace(y_min, y_max, n_points))
    plt.subplot(1, 1, 1)
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
    plt.xlabel(pair[0])
    plt.ylabel(pair[1])
    # Compute score 
    scores = clf.score(df_to_plot.values, labels.ravel())
    # Title has the two genes and the score
    plt.title(' vs '.join(pair) + ' - score: ' + str(round(scores.mean(), 2)))
    plt.legend(loc = 'best', fontsize = 10)
    plt.ioff()
    if path: 
        # REMEMBER THAT None == False. If path is None, the figure is not saved.
        # plt.savefig(path, dpi =300, bbox_inches='tight', format='png', transparent = False)
        plt.savefig(path, dpi =300, format='png', transparent = False)
    return None

In [32]:
N_figures = 200
pairs_to_plot_RNA = pd.read_csv(output_path + 'bivariate_scores_RNA.csv').sort_values(by='cv', ascending=False).head(N_figures).drop(['cv','train'], axis=1)
pairs_to_plot_PROTTISSUE = pd.read_csv(output_path + 'bivariate_scores_PROTTISSUE.csv').sort_values(by='cv', ascending=False).head(N_figures).drop(['cv','train'], axis=1)
pairs_to_plot_PROTSERUM = pd.read_csv(output_path + 'bivariate_scores_PROTSERUM.csv').sort_values(by='cv', ascending=False).head(N_figures).drop(['cv','train'], axis=1)

In [33]:
# MODIFY PATH HERE FOR DESIRED GROUP SELECTION
figures_path = '../figures/NEPBSvsNEAmAc/'
if not os.path.exists(figures_path):
   os.makedirs(figures_path)

In [34]:
if not os.path.exists(figures_path +'RNA'):
   os.makedirs(figures_path + 'RNA')
if not os.path.exists(figures_path +'PROTTISSUE'):
   os.makedirs(figures_path + 'PROTTISSUE')
if not os.path.exists(figures_path +'PROTSERUM'):
   os.makedirs(figures_path + 'PROTSERUM')


for i, pair in enumerate( tqdm(pairs_to_plot_RNA.values.tolist()) ):
   plot_pair_SVM(pair, df_rna.T, labels_rna, path = figures_path + 'RNA/' + str(i+1) + '_' + pair[0] + '_' + pair[1] + '.png', show=False)

for i, pair in enumerate( tqdm(pairs_to_plot_PROTTISSUE.values.tolist()) ):
   plot_pair_SVM(pair, df_prottissue.T, labels_prot, path = figures_path + 'PROTTISSUE/' + str(i+1) + '_' + pair[0] + '_' + pair[1] + '.png', show=False)

for i, pair in enumerate( tqdm(pairs_to_plot_PROTSERUM.values.tolist()) ):
   plot_pair_SVM(pair, df_protserum.T, labels_prot, path = figures_path + 'PROTSERUM/' + str(i+1) + '_' + pair[0] + '_' + pair[1] + '.png', show=False)

 10%|█         | 20/200 [00:20<03:07,  1.04s/it]C:\Users\fetsuruk\AppData\Local\Temp\ipykernel_26484\2191759271.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
100%|██████████| 200/200 [09:42<00:00,  2.91s/it]
